In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

#path = "/Users/Dipesh/OneDrive/Research_Engineer/Papers/M2EC/results/AWS_Google/"
path = "/Users/Dipesh/OneDrive/Research_Engineer/Papers/M2EC/results/AWS_all_Google/"

bandwidth ={}
latency  = {}

bw = []
lt = []

listUse=['frankfurt_google', 'london_google', 'mumbai_google', 'northvirginia_google', 'oregon_google', 'saopaulo_google', 'singapore_google', 'sydney_google', 'tokyo_google']


In [138]:
dirList = os.listdir(path)
dirList.remove(".DS_Store")
#for i in listUse:
   # for j in listUse: 
for i in dirList:
    for j in dirList: 
        if i is j:
            bw.append(0.01)
            lt.append(0.01)
            continue
            

        # bandwidth
        if "_aws" in j:
            location = path+"%s/combined_bandwidth_aws_%s_%s.txt"%(i,j,i)
        else:
            location = path+"%s/combined_bandwidth_google_%s_%s.txt"%(i,j,i)
        
        my_file = Path(location)
        if my_file.exists()==False:
            bw.append(0)
            lt.append(0)
            continue

        df_bandwidth = pd.read_csv(location, sep='\s+', skiprows=3, nrows=100, header=None)
        headers = ["0", "1", "time","time_unit","transfer","transfer_unit", "bitrate", "bitrate_unit", "retr", "cwnd", "cwnd_unit"]
        df_bandwidth.columns = headers     
        df_bandwidth.drop(['0', '1', 'time', 'time_unit','transfer', 'transfer_unit','cwnd', 'cwnd_unit', 'retr'], axis=1, inplace= True)
        val = df_bandwidth["bitrate"].mean()
        
        bandwidth[j+"_"+i]=val
        bw.append(val)
  
        # latency
        if "_aws" in j:
            location = path+"%s/combined_latency_aws_%s_%s.txt"%(i,j,i)
        else:
            location = path+"%s/combined_latency_google_%s_%s.txt"%(i,j,i)
        
        df_lat = pd.read_csv(location, sep='\s+', skiprows=1, nrows=100, header=None)

        headers_lat = ["0", "1", "2", "3", "4", "5", "latency", "unit"]
        df_lat.columns = headers_lat
        df_lat.drop(['0', '1', '2', '3','4', '5'], axis=1, inplace= True)        
        df_lat["latency"] = df_lat["latency"].map(lambda x:x.replace("time=",""))
        df_lat["latency"] = df_lat["latency"].astype('float')
        #print(df_lat["latency"].max())
        val_lat = df_lat["latency"].mean()
        
        latency[j+"_"+i]=val_lat
        lt.append(val_lat)

In [139]:
# bandwidth
arrayBw = np.array(bw)
#arrayBw= arrayBw.reshape(-1,len(dirList))
arrayBw= arrayBw.reshape(-1,len(dirList))

max_bd = np.max(arrayBw)
min_bd = np.amin(arrayBw[arrayBw>0.01])

arrayBw = (arrayBw-min_bd)/(max_bd-min_bd)
arrayBw = arrayBw.round(4)
#print(latency)

In [140]:
# latency
arrayLt = np.array(lt)
arrayLt = arrayLt.reshape(-1,len(dirList))

max_lt = np.max(arrayLt)
min_lt = np.amin(arrayLt[arrayLt>0.01])

arrayLt = 1-(arrayLt-min_lt)/(max_lt-min_lt)

arrayLt = arrayLt.round(4)

In [141]:
arrayLt = np.where(arrayLt==1.0041, 0, arrayLt)

In [142]:
combined = (arrayBw+arrayLt)/2
print(combined.shape)

(24, 24)


In [143]:
df = pd.DataFrame(combined)
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.50015,0.41510,-0.00185,0.44860,0.39865,-0.00185,0.27695,-0.00185,0.42855,0.62950,...,0.42925,0.38350,-0.00185,0.29870,0.25045,-0.00185,0.26175,-0.00185,0.33490,-0.00185
1,0.42655,0.50015,0.90955,0.62270,0.66940,0.60535,0.38910,0.07590,0.33615,0.46055,...,0.67075,0.24240,0.24410,0.19320,0.32995,0.13885,0.15975,0.16460,0.21870,0.21270
2,-0.00185,0.81400,0.50015,-0.00185,0.59940,0.80980,0.20380,0.06760,-0.00185,0.41745,...,-0.00185,0.22675,0.22960,-0.00185,0.14160,0.15725,0.14660,0.17330,0.20675,0.24245
3,0.45080,0.58345,-0.00185,0.50015,0.58360,-0.00185,0.37525,-0.00185,0.33780,0.42975,...,0.52720,0.29770,-0.00185,0.19550,0.30980,-0.00185,0.17745,-0.00185,0.24470,-0.00185
4,0.43715,0.65505,0.62055,0.74775,0.50015,0.99885,0.38360,0.08290,0.34790,0.46745,...,0.71215,0.25680,0.26700,0.19045,0.33160,0.16895,0.16600,0.18625,0.23280,0.25295
5,-0.00185,0.51660,0.54260,-0.00185,0.94330,0.50015,0.20385,0.08920,-0.00185,0.44025,...,-0.00185,0.24285,0.24585,-0.00185,0.15870,0.17295,0.16100,0.19265,0.22045,0.24175
6,0.27420,0.39510,0.21985,0.37995,0.39105,0.22195,0.50015,0.95285,0.20500,0.29540,...,0.40820,0.11055,0.05885,0.33885,0.48300,0.46735,0.20080,0.32300,0.35845,0.36975
7,-0.00185,0.05690,0.07720,-0.00185,0.07295,0.09120,0.91860,0.50015,-0.00185,0.15735,...,-0.00185,0.01535,0.01425,-0.00185,0.48430,0.48520,0.31140,0.36880,0.34065,0.36500
8,0.45580,0.34755,-0.00185,0.34690,0.36385,-0.00185,0.20985,-0.00185,0.50015,0.49180,...,0.36945,0.27690,-0.00185,0.36185,0.31190,-0.00185,0.34565,-0.00185,0.40260,-0.00185
9,-0.00185,0.45085,0.43585,-0.00185,0.49395,0.46665,0.28735,0.17460,-0.00185,0.50015,...,-0.00185,0.36580,0.37140,-0.00185,0.22365,0.26475,0.26905,0.28470,0.32825,0.34290


In [146]:
#df = pd.DataFrame(combined, index=dirList, columns=dirList)
df = pd.DataFrame(combined)
df.columns = dirList
new_column = dirList
df.insert(0, column="From", value=new_column)

df = df.round(4)
df = df.replace(-0.0018,0)
df = df.replace(0.5002,1)
#df.replace(-0.0160,0)
df


,From,central_aws,frankfurt_aws,frankfurt_google,ireland_aws,london_aws,london_google,mumbai_aws,mumbai_google,northcalifornia_aws,...,paris_aws,saopaulo_aws,saopaulo_google,seoul_aws,singapore_aws,singapore_google,sydney_aws,sydney_google,tokyo_aws,tokyo_google
0,central_aws,1.0000,0.4151,0.0000,0.4486,0.3986,0.0000,0.2769,0.0000,0.4286,...,0.4292,0.3835,0.0000,0.2987,0.2504,0.0000,0.2618,0.0000,0.3349,0.0000
1,frankfurt_aws,0.4266,1.0000,0.9096,0.6227,0.6694,0.6053,0.3891,0.0759,0.3361,...,0.6708,0.2424,0.2441,0.1932,0.3299,0.1388,0.1598,0.1646,0.2187,0.2127
2,frankfurt_google,0.0000,0.8140,1.0000,0.0000,0.5994,0.8098,0.2038,0.0676,0.0000,...,0.0000,0.2268,0.2296,0.0000,0.1416,0.1572,0.1466,0.1733,0.2068,0.2424
3,ireland_aws,0.4508,0.5834,0.0000,1.0000,0.5836,0.0000,0.3752,0.0000,0.3378,...,0.5272,0.2977,0.0000,0.1955,0.3098,0.0000,0.1774,0.0000,0.2447,0.0000
4,london_aws,0.4372,0.6550,0.6205,0.7478,1.0000,0.9988,0.3836,0.0829,0.3479,...,0.7122,0.2568,0.2670,0.1904,0.3316,0.1690,0.1660,0.1862,0.2328,0.2530
5,london_google,0.0000,0.5166,0.5426,0.0000,0.9433,1.0000,0.2038,0.0892,0.0000,...,0.0000,0.2428,0.2458,0.0000,0.1587,0.1730,0.1610,0.1927,0.2204,0.2418
6,mumbai_aws,0.2742,0.3951,0.2198,0.3800,0.3910,0.2220,1.0000,0.9528,0.2050,...,0.4082,0.1106,0.0588,0.3388,0.4830,0.4674,0.2008,0.3230,0.3584,0.3698
7,mumbai_google,0.0000,0.0569,0.0772,0.0000,0.0730,0.0912,0.9186,1.0000,0.0000,...,0.0000,0.0154,0.0142,0.0000,0.4843,0.4852,0.3114,0.3688,0.3406,0.3650
8,northcalifornia_aws,0.4558,0.3475,0.0000,0.3469,0.3638,0.0000,0.2098,0.0000,1.0000,...,0.3695,0.2769,0.0000,0.3618,0.3119,0.0000,0.3456,0.0000,0.4026,0.0000
9,northvirginia_aws,0.0000,0.4508,0.4358,0.0000,0.4940,0.4666,0.2874,0.1746,0.0000,...,0.0000,0.3658,0.3714,0.0000,0.2236,0.2648,0.2690,0.2847,0.3282,0.3429


In [147]:
#df.max(axis=1)

In [148]:
df.to_csv("combined.csv", sep=",")

In [149]:
new_df = pd.melt(df, id_vars=['From'], value_vars=df.columns.values.tolist()[1:],
    var_name='To', value_name='Value')

In [150]:
new_df.head()

,From,To,Value
0,central_aws,central_aws,1.0000
1,frankfurt_aws,central_aws,0.4266
2,frankfurt_google,central_aws,0.0000
3,ireland_aws,central_aws,0.4508
4,london_aws,central_aws,0.4372


In [152]:
new_df.to_csv("values.csv", sep=",")